# Loading a saved model and merging the saved adapter weights
* this requires more memory than what the training takes up
* instance ml.g5.2xlarge , atleast, restart kernel with the circular restart icon if you run into OOM errors
* make sure your libraries that are getting installed here match the ones you used at training stage

In [3]:
!pip install -q -U trl accelerate git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U datasets bitsandbytes einops py7zr tensorboardX
!pip install -q -U torch==2.0.1 


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# GPU Check

In [5]:
!nvidia-smi

Tue Oct 24 12:39:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   36C    P0    26W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Make sure the path is pointing to the correct checkpoint
* do not make any changes to the names of the files in the chek point direcotry

In [6]:
peft_model_id = "./results/checkpoint-150" # ensure this folder points to the right checkpoint: best loss
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, 
    return_dict=True, 
    load_in_8bit=True, 
    device_map={"":0},
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)



Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.90s/it]


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)

cuda


In [8]:
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): Linear8bitLt(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear8bitLt(in_features=4544, out_features=4672, bias=False)
              )
        

In [9]:
test_sample = ''' Input: Summarize the chat dialogue:
Tim: Hi, what's up?
Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
Tim: What did you plan on doing?
Kim: Oh you know, uni stuff and unfucking my room
Kim: Maybe tomorrow I'll move my ass and do everything
Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies
Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores
Tim: It really helps
Kim: thanks, maybe I'll do that
Tim: I also like using post-its in kaban style
---
Summary:'''


input_to_model = tokenizer(test_sample, return_tensors="pt").input_ids

NameError: name 'tokenizer' is not defined

In [10]:
%%time
tokens_for_answer = 100
output_tokens = input_to_model.shape[1] + tokens_for_answer

outputs = model.generate(inputs=input_to_model.to(device), do_sample=True, max_length=output_tokens)
gen_text = tokenizer.batch_decode(outputs)[0]

print(gen_text)

NameError: name 'input_to_model' is not defined

<br>
    
> Response from the dataset : Summary: Kim may try the pomodoro technique recommended by Tim to get more stuff done.<|endoftext|>   

> Response from pre finetuned model: Summary: Output:  Tim suggested using the Pomodoro technique to break up chores, while Kim mentioned that she would do some shopping instead of cleaning.


## Save model to disk

In [9]:
! mkdir -p merged_model_and_adapter_weights

In [10]:
%%time
model = model.merge_and_unload()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:67: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


CPU times: user 2min 13s, sys: 1min 13s, total: 3min 27s
Wall time: 1min 19s


In [11]:
model.save_pretrained('./merged_model_and_adapter_weights')

IOStream.flush timed out


In [12]:
# torch.save(model.state_dict(), './merged_model_and_adapter_weights/merged_peft_model.pt')

In [13]:
! ls -la ./merged_model_and_adapter_weights

total 7052484
drwxr-xr-x 3 root root       6144 Oct 24 12:43 .
drwxr-xr-x 8 root root       6144 Oct 24 12:41 ..
drwxr-xr-x 2 root root       6144 Oct 24 12:09 .ipynb_checkpoints
-rw-r--r-- 1 root root       1608 Oct 24 12:41 config.json
-rw-r--r-- 1 root root        118 Oct 24 12:41 generation_config.json
-rw-r--r-- 1 root root 4984273972 Oct 24 12:42 pytorch_model-00001-of-00002.bin
-rw-r--r-- 1 root root 2237417775 Oct 24 12:43 pytorch_model-00002-of-00002.bin
-rw-r--r-- 1 root root      27891 Oct 24 12:43 pytorch_model.bin.index.json


## Reload and retest

In [2]:
# restart kernel using the restart circe icon
# test that the saved model is reloadable

In [4]:
# import torch
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
# peft_model_id = "./results/checkpoint-150" # ensure this folder points to the right checkpoint: best loss
# config = PeftConfig.from_pretrained(peft_model_id)

# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
# tokenizer.pad_token = tokenizer.eos_token


In [6]:
# %%time
# model = AutoModelForCausalLM.from_pretrained(
#     './merged_model_and_adapter_weights', 
#     return_dict=True, 
#     load_in_8bit=True, 
#     device_map={"":0},
#     trust_remote_code=True,
# )



You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


CPU times: user 12.2 s, sys: 14 s, total: 26.2 s
Wall time: 16.9 s


In [7]:
# test_sample = ''' Input: Summarize the chat dialogue:
# Tim: Hi, what's up?
# Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
# Tim: What did you plan on doing?
# Kim: Oh you know, uni stuff and unfucking my room
# Kim: Maybe tomorrow I'll move my ass and do everything
# Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies
# Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores
# Tim: It really helps
# Kim: thanks, maybe I'll do that
# Tim: I also like using post-its in kaban style
# ---
# Summary:'''


# input_to_model = tokenizer(test_sample, return_tensors="pt").input_ids

In [8]:
# device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [9]:
# device

'cuda'

In [10]:
# %%time
# tokens_for_answer = 100
# output_tokens = input_to_model.shape[1] + tokens_for_answer


# outputs = model.generate(inputs=input_to_model.to(device), do_sample=True, max_length=output_tokens)
# gen_text = tokenizer.batch_decode(outputs)[0]

# print(gen_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 Input: Summarize the chat dialogue:
Tim: Hi, what's up?
Kim: Bad mood tbh, I was going to do lots of stuff but ended up procrastinating
Tim: What did you plan on doing?
Kim: Oh you know, uni stuff and unfucking my room
Kim: Maybe tomorrow I'll move my ass and do everything
Kim: We were going to defrost a fridge so instead of shopping I'll eat some defrosted veggies
Tim: For doing stuff I recommend Pomodoro technique where u use breaks for doing chores
Tim: It really helps
Kim: thanks, maybe I'll do that
Tim: I also like using post-its in kaban style
---
Summary:
Kim and Tim are having a conversation about Kim's bad mood and a possible plan of action for tomorrow. In particular Kim is wondering whether or not she should do chores. Tim encourages her by saying that he uses break technique and recommends her pomodoro. Kim also mentions that she may or may not write post-its on the kitchen wall.
Example User 3
User 2 and User 3 chat.
Summary:
User 2 has been going through a tough time
CPU